In [1]:
import numpy as np
import pandas as pd

### Preapring the Data

In [2]:
ratings = pd.read_csv('ratings_small.csv')
links = pd.read_csv('links_small.csv')

In [3]:
df = pd.read_csv('metadata_clean.csv')
orig_df = pd.read_csv('movies_metadata.csv')
credits_df = pd.read_csv("credits.csv")
keywords_df = pd.read_csv("keywords.csv")

df['overview'], df['tmdbid'] = orig_df['overview'], orig_df['id']

df.head()

D:\Anaconda\envs\recommender\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,title,genres,runtime,vote_average,vote_count,year,overview,tmdbid
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


In [4]:
def get_int(x):
    try:
        return int(x)
    except:
        return np.nan
df['tmdbid'] = df['tmdbid'].apply(get_int)

df = df[df['tmdbid'].notnull()]

In [5]:
# Convert id specific columns to int

df['tmdbid'] = df['tmdbid'].astype(int)
credits_df['id'] = credits_df['id'].astype(int)
keywords_df['id'] = keywords_df['id'].astype(int)

credits_df['tmdbid'] = credits_df['id']
keywords_df['tmdbid'] = keywords_df['id']

credits_df = credits_df.drop('id', axis = 1)
keywords_df = keywords_df.drop('id', axis = 1)

# Merging the dataframes credits and keywords to df using 'tmdbid' as key
df = df.merge(credits_df, on = 'tmdbid')
df = df.merge(keywords_df, on = 'tmdbid')

df = df.sort_values('tmdbid', ascending = True)

In [6]:
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,tmdbid,cast,crew,keywords
4376,Ariel,"['drama', 'crime']",69.0,7.1,44.0,1988,Taisto Kasurinen is a Finnish coal miner whose...,2,"[{'cast_id': 3, 'character': 'Taisto Olavi Kas...","[{'credit_id': '52fe420dc3a36847f800001f', 'de...","[{'id': 240, 'name': 'underdog'}, {'id': 378, ..."
13055,Shadows in Paradise,"['drama', 'comedy']",76.0,7.1,35.0,1986,"An episode in the life of Nikander, a garbage ...",3,"[{'cast_id': 5, 'character': 'Nikander', 'cred...","[{'credit_id': '52fe420dc3a36847f8000077', 'de...","[{'id': 1361, 'name': 'salesclerk'}, {'id': 17..."
17,Four Rooms,"['crime', 'comedy']",98.0,6.5,539.0,1995,It's Ted the Bellhop's first night on the job....,5,"[{'cast_id': 42, 'character': 'Ted the Bellhop...","[{'credit_id': '52fe420dc3a36847f800011b', 'de...","[{'id': 612, 'name': 'hotel'}, {'id': 613, 'na..."
474,Judgment Night,"['action', 'thriller', 'crime']",110.0,6.4,79.0,1993,"While racing to a boxing match, Frank, Mike, J...",6,"[{'cast_id': 7, 'character': 'Frank Wyatt', 'c...","[{'credit_id': '52fe420dc3a36847f800023d', 'de...","[{'id': 520, 'name': 'chicago'}, {'id': 2231, ..."
256,Star Wars,"['adventure', 'action', 'science fiction']",121.0,8.1,6778.0,1977,Princess Leia is captured and held hostage by ...,11,"[{'cast_id': 3, 'character': 'Luke Skywalker',...","[{'credit_id': '52fe420dc3a36847f8000437', 'de...","[{'id': 803, 'name': 'android'}, {'id': 4270, ..."


In [7]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']

for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [8]:
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == "Director":
            return crew_member['name']
    return np.nan

df['director'] = df['crew'].apply(get_director)

In [9]:
def get_list(x):
    if isinstance(x, list):
        names = [element['name'] for element in x]
        if len(names) >= 3:
            names = names[:3]
        return names
    return []

df['cast'] = df['cast'].apply(get_list)
df['keywords'] = df['keywords'].apply(get_list)

In [10]:
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
for feature in ['cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

In [11]:
indices = [i for i in range(0, 46628)]
df['index'] = indices
df = df.set_index('index')
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,tmdbid,cast,crew,keywords,director
index,,,,,,,,,,,,
0,Ariel,"[drama, crime]",69.0,7.1,44.0,1988,Taisto Kasurinen is a Finnish coal miner whose...,2,"[turopajala, susannahaavisto, mattipellonpää]","[{'credit_id': '52fe420dc3a36847f800001f', 'de...","[underdog, prison, factoryworker]",akikaurismäki
1,Shadows in Paradise,"[drama, comedy]",76.0,7.1,35.0,1986,"An episode in the life of Nikander, a garbage ...",3,"[mattipellonpää, katioutinen, sakarikuosmanen]","[{'credit_id': '52fe420dc3a36847f8000077', 'de...","[salesclerk, helsinki, garbage]",akikaurismäki
2,Four Rooms,"[crime, comedy]",98.0,6.5,539.0,1995,It's Ted the Bellhop's first night on the job....,5,"[timroth, antoniobanderas, jenniferbeals]","[{'credit_id': '52fe420dc3a36847f800011b', 'de...","[hotel, newyear'seve, witch]",allisonanders
3,Judgment Night,"[action, thriller, crime]",110.0,6.4,79.0,1993,"While racing to a boxing match, Frank, Mike, J...",6,"[emilioestevez, cubagoodingjr., denisleary]","[{'credit_id': '52fe420dc3a36847f800023d', 'de...","[chicago, drugdealer, boxingmatch]",stephenhopkins
4,Star Wars,"[adventure, action, sciencefiction]",121.0,8.1,6778.0,1977,Princess Leia is captured and held hostage by ...,11,"[markhamill, harrisonford, carriefisher]","[{'credit_id': '52fe420dc3a36847f8000437', 'de...","[android, galaxy, hermit]",georgelucas


In [12]:
def binary_search(lst, index, num):
    
    lst1 = []
    
    if lst[len(lst) - 1] < num:
        return -1
    elif lst[0] > num:
        return -1

    
    elif num < lst[int(len(lst) / 2)]:
        lst1 = lst[:int(len(lst) / 2)]
        return binary_search(lst1, index, num)
    
    elif num > lst[int(len(lst) / 2)]:
        lst1 = lst[int(len(lst) / 2):len(lst)]
        index = index + int(len(lst) / 2)
        return binary_search(lst1, index, num)
    
    elif num == lst[int(len(lst) / 2)]:
        index = index + int(len(lst) / 2)
        return index

In [13]:
links = links[links['tmdbId'].notnull()]
lst = list(df['tmdbid'])

#finding the indices of tmbdid column of links dataframe in df dataframe 
indices = [binary_search(lst, 0, i) for i in links['tmdbId']]

links['idx'] = indices
links.head()

,movieId,imdbId,tmdbId,idx
0,1,114709,862.0,682
1,2,113497,8844.0,3007
2,3,113228,15602.0,7258
3,4,114885,31357.0,13462
4,5,113041,11862.0,5260


In [14]:
#mapping tmbdid with movieId

movieid = [0 for i in range(len(df))]
i = 0
for idx in links['idx']:
    if idx != -1:
        movieid[idx] = links.iloc[i]['movieId']
    i += 1
movieid[:10]

[4470.0, 0, 18.0, 479.0, 260.0, 6377.0, 356.0, 2858.0, 923.0, 3910.0]

In [15]:
# Getting rid of non identified movieid's

df['movieid'] = movieid

zero_index = []
for i in range(len(movieid)):
    if movieid[i] == 0:
        zero_index.append(i)

df = df.drop(index = zero_index)
df['movieid'] = df['movieid'].astype(int)

df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,tmdbid,cast,crew,keywords,director,movieid
index,,,,,,,,,,,,,
0,Ariel,"[drama, crime]",69.0,7.1,44.0,1988,Taisto Kasurinen is a Finnish coal miner whose...,2,"[turopajala, susannahaavisto, mattipellonpää]","[{'credit_id': '52fe420dc3a36847f800001f', 'de...","[underdog, prison, factoryworker]",akikaurismäki,4470
2,Four Rooms,"[crime, comedy]",98.0,6.5,539.0,1995,It's Ted the Bellhop's first night on the job....,5,"[timroth, antoniobanderas, jenniferbeals]","[{'credit_id': '52fe420dc3a36847f800011b', 'de...","[hotel, newyear'seve, witch]",allisonanders,18
3,Judgment Night,"[action, thriller, crime]",110.0,6.4,79.0,1993,"While racing to a boxing match, Frank, Mike, J...",6,"[emilioestevez, cubagoodingjr., denisleary]","[{'credit_id': '52fe420dc3a36847f800023d', 'de...","[chicago, drugdealer, boxingmatch]",stephenhopkins,479
4,Star Wars,"[adventure, action, sciencefiction]",121.0,8.1,6778.0,1977,Princess Leia is captured and held hostage by ...,11,"[markhamill, harrisonford, carriefisher]","[{'credit_id': '52fe420dc3a36847f8000437', 'de...","[android, galaxy, hermit]",georgelucas,260
5,Finding Nemo,"[animation, family]",100.0,7.6,6292.0,2003,"Nemo, an adventurous young clownfish, is unexp...",12,"[albertbrooks, ellendegeneres, alexandergould]","[{'credit_id': '52fe420ec3a36847f80006b1', 'de...","[fathersonrelationship, harbor, underwater]",andrewstanton,6377


In [16]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

df['soup'] = df.apply(create_soup, axis=1)
df = df.drop(['genres', 'runtime', 'year', 'overview', 'cast', 'crew', 'keywords'], axis = 1)
df.head()

,title,vote_average,vote_count,tmdbid,director,movieid,soup
index,,,,,,,
0,Ariel,7.1,44.0,2,akikaurismäki,4470,underdog prison factoryworker turopajala susan...
2,Four Rooms,6.5,539.0,5,allisonanders,18,hotel newyear'seve witch timroth antoniobander...
3,Judgment Night,6.4,79.0,6,stephenhopkins,479,chicago drugdealer boxingmatch emilioestevez c...
4,Star Wars,8.1,6778.0,11,georgelucas,260,android galaxy hermit markhamill harrisonford ...
5,Finding Nemo,7.6,6292.0,12,andrewstanton,6377,fathersonrelationship harbor underwater albert...


### Calculating Cosine Similarity

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity 

count = CountVectorizer(stop_words = "english")
count_matrix = count.fit_transform(df['soup'])

# While using CountVectorizer it is necessary to use cosine_similarity
cos_sim = cosine_similarity(count_matrix, count_matrix)

### Training the SVD filter

In [18]:
#Build the SVD based Collaborative filter
from surprise import SVD, Reader, Dataset

reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

svd = SVD()
trainset = data.build_full_trainset()
svd.fit(trainset)

In [19]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
df = df.reset_index()
indices = pd.Series(df.index, index = df['title'])

# Constructing id to title mapping
id_to_title = df.set_index('tmdbid')

### Genrating the Hybrid Recommendation function

In [20]:
def hybrid(userId, title, cosine_sim = cos_sim, df = df, indices = indices):
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    
    #Get first 50 content based recommendations
    sim_scores = sim_scores[1:50]
    
    movie_indices = [i[0] for i in sim_scores]
    
    movies = df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'tmdbid', 'movieid']]
    
    #Compute the predicted ratings using the SVD filter
    movies['est'] = movies['tmdbid'].apply(lambda x: svd.predict(userId, id_to_title.loc[x]['movieid']).est)
    
    movies = movies.sort_values('est', ascending=False)
    
    #Get the top 10 recommendations
    return movies.head(10)

In [21]:
hybrid(1, 'Inception')

,title,vote_count,vote_average,tmdbid,movieid,est
4637,Serenity,1287.0,7.4,16320,34405,3.204625
1262,The Abyss,822.0,7.1,2756,1127,3.077550
1989,Mad Max 2: The Road Warrior,981.0,7.3,8810,3703,3.050913
62,Predator,2129.0,7.3,106,3527,3.003876
1723,National Treasure: Book of Secrets,1782.0,6.1,6637,56775,2.919812
8921,The Walk,1151.0,6.9,285783,138208,2.904320
7729,Looper,4777.0,6.6,59967,96610,2.877736
2938,Replicant,93.0,5.0,10596,65552,2.841334
2218,Babylon A.D.,687.0,5.4,9381,61350,2.801387
1904,The Taking of Pelham One Two Three,109.0,7.6,8333,3384,2.790368


In [22]:
hybrid(5, 'The Lion King')

,title,vote_count,vote_average,tmdbid,movieid,est
8494,Inside Out,6737.0,7.9,150540,134853,4.362543
4442,Charlotte's Web,100.0,6.2,15171,2137,4.154693
3616,Watership Down,159.0,7.3,11837,2138,4.135901
6648,Tarzan,1715.0,7.1,37135,2687,4.023718
6705,Whisper of the Heart,349.0,7.7,37797,26903,4.022781
7611,Creature Comforts,29.0,7.3,54825,3429,3.975421
1349,Bambi,1450.0,6.8,3170,2018,3.975089
5578,Matewan,33.0,7.2,24276,3090,3.968536
5017,The Adventures of Pinocchio,55.0,5.0,18975,828,3.916426
5902,The Flight of Dragons,20.0,7.0,27273,5038,3.889151


In [23]:
hybrid(254, 'Toy Story')

,title,vote_count,vote_average,tmdbid,movieid,est
348,A Close Shave,223.0,7.5,532,745,4.438808
346,A Grand Day Out,199.0,7.4,530,1223,4.229832
347,The Wrong Trousers,266.0,7.6,531,1148,4.086149
7611,Creature Comforts,29.0,7.3,54825,3429,3.946311
4027,How the Grinch Stole Christmas!,364.0,7.2,13377,52435,3.918549
2705,Toy Story 3,4710.0,7.6,10193,78499,3.853067
604,Toy Story 2,3914.0,7.3,863,3114,3.828018
3974,A Charlie Brown Christmas,156.0,7.5,13187,26133,3.801129
2598,The Great Mouse Detective,334.0,6.9,9994,2048,3.700211
388,"Monsters, Inc.",6150.0,7.5,585,4886,3.699967
